In [1]:
import boto3
boto3.client('sts').get_caller_identity()

{'UserId': 'AROAV52AN3HJJIZVUDZ2F:SageMaker',
 'Account': '407620147666',
 'Arn': 'arn:aws:sts::407620147666:assumed-role/role-ap-segemaker-execution-setopdata2.0-t/SageMaker',
 'ResponseMetadata': {'RequestId': '4701395d-23a0-4e73-8b11-c9b55554fcbd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4701395d-23a0-4e73-8b11-c9b55554fcbd',
   'content-type': 'text/xml',
   'content-length': '467',
   'date': 'Wed, 09 Aug 2023 15:31:46 GMT'},
  'RetryAttempts': 0}}

In [2]:
# Create session using your current creds
boto_sts=boto3.client('sts')

# Request to assume the role like this, the ARN is the Role's ARN from 
# the other account you wish to assume. Not your current ARN.
stsresponse = boto_sts.assume_role(
    RoleArn="arn:aws:iam::354925255288:role/test",
    RoleSessionName='newsession'
)

# Save the details from assumed role into vars
newsession_id = stsresponse["Credentials"]["AccessKeyId"]
newsession_key = stsresponse["Credentials"]["SecretAccessKey"]
newsession_token = stsresponse["Credentials"]["SessionToken"]


In [3]:
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [4]:
sm_client = boto3.client(
    'sagemaker',
    region_name='us-east-1',
    aws_access_key_id=newsession_id,
    aws_secret_access_key=newsession_key,
    aws_session_token=newsession_token
)

In [5]:
smr_client = boto3.client(
    'sagemaker-runtime',
    region_name='us-east-1',
    aws_access_key_id=newsession_id,
    aws_secret_access_key=newsession_key,
    aws_session_token=newsession_token
)

In [6]:
endpoint_name = 'huggingface-pytorch-tgi-inference-2023-08-09-14-15-39-283'

In [7]:
import sagemaker

In [8]:
sagemaker_session = sagemaker.session.Session(
    sagemaker_client=sm_client, 
    sagemaker_runtime_client=smr_client)

In [9]:
from sagemaker import serializers

In [10]:
from sagemaker import deserializers

In [11]:
llm = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name, 
    sagemaker_session=sagemaker_session, 
    serializer=serializers.JSONSerializer(),
    deserializers=deserializers.JSONDeserializer())

In [12]:
def build_llama2_prompt(messages):
    startPrompt = "<s>[INST] "
    endPrompt = " [/INST]"
    conversation = []
    for index, message in enumerate(messages):
        if message["role"] == "system" and index == 0:
            conversation.append(f"<<SYS>>\n{message['content']}\n<</SYS>>\n\n")
        elif message["role"] == "user":
            conversation.append(message["content"].strip())
        else:
            conversation.append(f" [/INST] {message.content}</s><s>[INST] ")

    return startPrompt + "".join(conversation) + endPrompt



In [21]:
import re
import json


class Llama270BGenerater:
    
    def __init__(self, llm, include_inputs=False):
        self.llm = llm
        self.include_inputs = include_inputs

        
    def predict(self, prompt):

        messages = [
          { "role": "system","content": prompt}
        ]


        # define question and add to messages
        # instruction = "What are some cool ideas to do in the summer?"
        # messages.append({"role": "user", "content": instruction})
        _prompt = self.build_llama2_prompt(messages)

        payload = {
          "inputs":  _prompt,
          "parameters": {
            "do_sample": True,
            "top_p": 0.6,
            "temperature": 0.01,
            "top_k": 50,
            "max_new_tokens": 1000,
            "repetition_penalty": 1.03,
            "stop": ["</s>"]
          }
        }
        
        # chat = llm.predict({"inputs": json.dumps(prompt)})
        chat = self.llm.predict(payload)
        
        ret = json.loads(chat)[0]["generated_text"]
        
        if not self.include_inputs:
            ret = ret[len(_prompt):]

        return ret
    
    @staticmethod
    def build_llama2_prompt(messages):
        startPrompt = "<s>[INST] "
        endPrompt = " [/INST]"
        conversation = []
        for index, message in enumerate(messages):
            if message["role"] == "system" and index == 0:
                conversation.append(f"<<SYS>>\n{message['content']}\n<</SYS>>\n\n")
            elif message["role"] == "user":
                conversation.append(message["content"].strip())
            else:
                conversation.append(f" [/INST] {message.content}</s><s>[INST] ")

        return startPrompt + "".join(conversation) + endPrompt



JSON_RE = re.compile(r'(\{.*\})')

class JsonExtract:
    
    def reply2result(self, reply: str) -> dict:

        json_str = JSON_RE.findall(reply.replace('\n', ' '))[0]
        ret = json.loads(json_str)
        return ret

    def result2products(self, result: dict, target_key='product_name') -> list[str]:
        assert target_key in result
        assert isinstance(result[target_key], list)

        ret = result[target_key]

        return ret

    def reply2products(self, reply:str) -> list[str]:
        
        result = self.reply2result(reply)
        ret = self.result2products(result)


        return ret

In [22]:
sample_generator = Llama270BGenerater(llm)

In [23]:
sample_generator.predict('hello!')

" Hello! It's nice to meet you. Is there something I can help you with or would you like to chat for a bit?"

In [31]:
instruction = '''
Please generate {n_samples} product names for {category} category.
1. Return the results in JSON format with the following key: "product_name".
2. Example product_name: {samples} .
3. Do not repeat inputs in the output.
4. Replied answer should be as diverse as possible.
5. Do not repeat answers.
6. Reply in Taiwan Chinese.
7. values with product_name is a list of product names.

The json result is: 
'''


class SampleAugumentation(JsonExtract):
    
    def __init__(self, sample_generator, instruction):
        self.sample_generator = sample_generator
        self.instruction = instruction
        
    def _sample_generate(self, category:str, samples:list, n_samples:int=10, debug=False):
        
        n_provided_samples = len(samples)
        
        samples = list(map(lambda x: '"{}",'.format(x.replace('"', '')), samples))
        samples = ' '.join(samples)
        
        prompt = self.instruction.format(category=category, samples=samples, n_samples=n_samples+n_provided_samples)
        response = self.sample_generator.predict(prompt)
        content = response
        # content = response.to_dict()['choices'][0]['message']['content']
        
        complete = prompt+content
        
        print('[debug]', 'prompt', prompt)
        print('[debug]', 'content', content)
        
        ret = self.reply2products(complete)[n_provided_samples:]
        
        return ret
    
    def sample_generate(self, category:str, samples:list, n_samples:int=10):
        debug = False
        while True:
            try:
                return self._sample_generate(category=category, samples=samples, n_samples=n_samples, debug=debug)
            except Exception as e:
                debug = True
                print(e)

In [32]:
sa = SampleAugumentation(sample_generator, instruction)

In [33]:
with open('workflow_asset/catogory_sample.json', 'r') as f:
    config = json.loads(f.read())

In [34]:
n_samples = 11
generated = dict()
for k, v in config.items():
    gen_samples = sa.sample_generate(category=k, samples=v, n_samples=n_samples)
    generated[k] = gen_samples
    
    print(k)
    print(v)
    print(gen_samples)
    
    print('----------')
    

[debug] prompt 
Please generate 13 product names for 現做咖啡飲品 category.
1. Return the results in JSON format with the following key: "product_name".
2. Example product_name: "全家特濃拿鐵", "西西里風味檸檬氣泡咖啡", .
3. Do not repeat inputs in the output.
4. Replied answer should be as diverse as possible.
5. Do not repeat answers.
6. Reply in Taiwan Chinese.
7. values with product_name is a list of product names.

The json result is: 

[debug] content  {
"product_name": [
"珍珠奶茶",
"迷你拿鐵",
"冰霜咖啡",
"豆漿奶粉咖啡",
"綠茶 latte",
"卡布奇諾",
"檸檬綠茶",
"椰奶咖啡",
"芋頭咖啡",
"奶粉綠茶",
"紅豆奶茶",
"咖啡奶粉",
"拿鐵奶粉"
]
}
現做咖啡飲品
['全家特濃拿鐵', '西西里風味檸檬氣泡咖啡']
['冰霜咖啡', '豆漿奶粉咖啡', '綠茶 latte', '卡布奇諾', '檸檬綠茶', '椰奶咖啡', '芋頭咖啡', '奶粉綠茶', '紅豆奶茶', '咖啡奶粉', '拿鐵奶粉']
----------
[debug] prompt 
Please generate 13 product names for 罐裝/瓶裝咖啡 category.
1. Return the results in JSON format with the following key: "product_name".
2. Example product_name: "《UCC》無糖咖啡飲料185g", "SANGARIA 圓潤咖啡飲料-拿鐵 (280gx24入)", .
3. Do not repeat inputs in the output.
4. Replied answer shou

In [37]:
with open('workflow_asset/catogory_sample_generate_llama270b.json', 'w') as f:
    f.write((json.dumps(generated)))